In [7]:
from configs import TestConfig
import torch
import jieba
import thulac
from utils import text_process, seg_char, BEGIN, SINGLE
thu = thulac.thulac(seg_only=True)

Model loaded succeed


In [2]:
configs = TestConfig()
model = configs.model
tokenizer = configs.tokenizer


Model loaded succeed


In [13]:
def bert_seg(model, tokenizer, sentence):
    """Bert 中文分词

    Args:
        sentence (str): 单个句子.
    Returns:
        word_list (list [str]): 分词结果.
    """
    sentence = text_process(sentence, is_tradition=True)
    char_list = seg_char(sentence)
    inputs = tokenizer(
        char_list,
        return_tensors="pt",
        is_split_into_words=True
    )
    output = model(inputs["input_ids"], inputs["attention_mask"])
    # [batch_size, src_len, 4]
    output = torch.argmax(output, dim=-1)[:, 1: -1].tolist()[0]
    char_list = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])[1: -1]
    idxes = [idx for idx, l in enumerate(output) if l == BEGIN or l == SINGLE] + [None]
    # print(idxes)
    word_list = ["".join(char_list[idxes[i]: idxes[i+1]]) for i in range(len(idxes)-1)]
    return word_list, char_list, output

In [15]:
sentence = "Bert可以用于中文分词"
word_list, char_list, output = bert_seg(model, tokenizer, sentence)
print(" ".join(word_list))
print(" ".join(jieba.cut(sentence)))
print(" ".join([x[0] for x in thu.cut(sentence)]))


be##rt 可以 用 于 中文 分词
Bert 可以 用于 中文 分词
Bert 可以 用于 中文 分词


In [16]:
sentence = "中国青年报发表文章《做题家们的怒气为何要往丁真身上撒》"
word_list, char_list, output = bert_seg(model, tokenizer, sentence)
print(" ".join(word_list))
print(" ".join(jieba.cut(sentence)))
print(" ".join([x[0] for x in thu.cut(sentence)]))

中国青年报 发表 文章 《 做题家 们 的 怒气 为何 要 往 丁真 身上 撒 》
中国青年报 发表文章 《 做题 家们 的 怒气 为何 要往 丁真 身上 撒 》
中国 青年报 发表 文章 《 做 题 家 们 的 怒气 为何 要 往 丁真 身上 撒 》


In [21]:
sentence = "内卷、打工人、做题家、小丑竟是我自己都是近年来较火的自嘲梗"
word_list, char_list, output = bert_seg(model, tokenizer, sentence)
print(" / ".join(word_list))
print(" / ".join(jieba.cut(sentence)))
print(" / ".join([x[0] for x in thu.cut(sentence)]))

内卷 / 、 / 打工人 / 、 / 做 / 题家 / 、 / 小丑 / 竟是 / 我 / 自己 / 都 / 是 / 近年 / 来 / 较 / 火 / 的 / 自嘲梗
内 / 卷 / 、 / 打 / 工人 / 、 / 做题 / 家 / 、 / 小丑 / 竟是 / 我 / 自己 / 都 / 是 / 近年来 / 较火 / 的 / 自嘲 / 梗
内卷 / 、 / 打工 / 人 / 、 / 做 / 题 / 家 / 、 / 小丑 / 竟是 / 我 / 自己 / 都 / 是 / 近年 / 来 / 较 / 火 / 的 / 自嘲梗
